# Fine-tuning Llama 3 on Riddle-ification

For our fine-tuning use-case, we'll leveraging [meta-llama/Meta-Llama-3-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3-8B-Instruct) on a summarization task in a particular style for a particular domain!



# Fine-tuning Example

We'll start, as we always do, with some dependencies!

In [ ]:
!pip install -qU transformers peft trl accelerate bitsandbytes datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 70.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.2/245.2 kB 24.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 40.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.0/102.0 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 36.5 MB/s eta 0:00:00


Next, let's set-up some data!

## Data

We'll be using a [dataset](https://huggingface.co/datasets/riddle_sense) that is adept at answering riddles for this version of the model.

This dataset contains the following:

- An Answer key
- A Question
- Multiple Choice questions

We'll start by grabbing our dataset from Hugging Face!

In [ ]:
from datasets import load_dataset

riddle_dataset = load_dataset("riddle_sense")

Let's see how many items we're working with in our dataset.

> NOTE: Keep in mind that this is a relatively simple example meant to showcase fine-tuning - in practice, we'd want to use somewhere in the neighbourhood of ~500-50,000 examples.

In [ ]:
riddle_dataset

DatasetDict({
    train: Dataset({
        features: ['answerKey', 'question', 'choices'],
        num_rows: 3510
    })
    validation: Dataset({
        features: ['answerKey', 'question', 'choices'],
        num_rows: 1021
    })
    test: Dataset({
        features: ['answerKey', 'question', 'choices'],
        num_rows: 1184
    })
})

Let's look at an example of our original text and summary!

In [ ]:
print(f"Question: {riddle_dataset['train'][0]['question']}\n\Choices: {riddle_dataset['train'][0]['choices']}\nAnswer: {riddle_dataset['train'][0]['answerKey']}")

Question: A man is incarcerated in prison, and as his punishment he has to carry a one tonne bag of sand backwards and forwards across a field the size of a football pitch.  What is the one thing he can put in it to make it lighter?
\Choices: {'label': ['A', 'B', 'C', 'D', 'E'], 'text': ['throw', 'bit', 'gallon', 'mouse', 'hole']}
Answer: E


Now, we mentioned earlier we were going to fine-tune meta-llama/Meta-Llama-3-8B-Instruct, which is important for our next step: Creating the instruction format.

Let's take a look at the instructions (so meta) to generate an instruction prompt from the [repository](https://github.com/meta-llama/llama3?tab=readme-ov-file#instruction-tuned-models)


> The fine-tuned models were trained for dialogue applications. To get the expected features and performance for them, a specific formatting defined in [`ChatFormat`](https://github.com/meta-llama/llama3/blob/main/llama/tokenizer.py#L202) needs to be followed: The prompt begins with a `<|begin_of_text|>` special token, after which one or more messages follow. Each message starts with the `<|start_header_id|>` tag, the role `system`, `user` or `assistant`, and the `<|end_header_id|>` tag. After a double newline "\n\n" the contents of the message follow. The end of each message is marked by the `<|eot_id|>` token.

Let's look at an example of how we might format our instruction - and then reproduce that in code.

```python
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Please answer the following multiple-choice riddle by selecting the correct choice. Only return the letter choice.<|eot_id|><|start_header_id|>user<|end_header_id|>

A man is incarcerated in prison, and as his punishment he has to carry a one tonne bag of sand backwards and forwards across a field the size of a football pitch.  What is the one thing he can put in it to make it lighter?

A: throw
B: bit
C: gallon
D: mouse
E: hole<|eot_id|><|start_header_id|>assistant<|end_header_id|>

E<|eot_id|>
```

In [ ]:
INSTRUCTION_PROMPT_TEMPLATE = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Please answer the following multiple-choice riddle by selecting the correct choice.<|eot_id|><|start_header_id|>user<|end_header_id|>

{question}\n\nA: {choice_a}\nB: {choice_b}\nC: {choice_c}\nD: {choice_d}\nE: {choice_e}\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

RESPONSE_TEMPLATE = """\
{answer}<|eot_id|><|end_of_text|>"""

Now we can create a helper function that will convert our dataset row into the above prompt!

In [ ]:
def create_instruction(sample, return_response=True):
  prompt = INSTRUCTION_PROMPT_TEMPLATE.format(
      question=sample["question"],
      choice_a=sample["choices"]["text"][0],
      choice_b=sample["choices"]["text"][1],
      choice_c=sample["choices"]["text"][2],
      choice_d=sample["choices"]["text"][3],
      choice_e=sample["choices"]["text"][4],
  )

  if return_response:
    prompt += RESPONSE_TEMPLATE.format(answer=sample["answerKey"])

  return prompt

Let's try it out!

In [ ]:
create_instruction(riddle_dataset["train"][0])

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nPlease answer the following multiple-choice riddle by selecting the correct choice. Only return the letter choice.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nA man is incarcerated in prison, and as his punishment he has to carry a one tonne bag of sand backwards and forwards across a field the size of a football pitch.  What is the one thing he can put in it to make it lighter?\n\nA: throw\nB: bit\nC: gallon\nD: mouse\nE: hole\n<|eot_id|><|start_header_id|>assistant<|end_header_id|>E<|eot_id|><|end_of_text|>'

## Loading Our Model

Now we can move onto loading our model!

We're going to be dependent on two major technologies to allow us to train our model with <=16GB GPU RAM.

1. Quantization
2. LoRA

> NOTE: We've done some events on [LoRA](https://www.youtube.com/watch?v=kV8yXIUC5_4&list=PLrSHiQgy4VjGMzyXsSlvN-TjPaqFFsAGP&index=4) and [QLoRA](https://www.youtube.com/watch?v=XOb-djcw6hs&list=PLrSHiQgy4VjGMzyXsSlvN-TjPaqFFsAGP&index=5) for deeper dives into those respective technologies

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

We'll load our tokenizer and do a few pre-processing steps to prepare it for training.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
riddle_dataset["test"][0]["question"]

'My life can be measured in hours. I serve by being devoured. Thin, I am quick. Fat, I am slow. Wind is my foe. What am I?'

In [ ]:
for text, label in zip(riddle_dataset["test"][0]["choices"]["text"], riddle_dataset["test"][0]["choices"]["label"]):
  print(f"{label} : {text}")

A : paper
B : candle
C : lamp
D : clock
E : worm


In [ ]:
from transformers import pipeline

base_model_pipe = pipeline("text-generation", model, tokenizer=tokenizer, max_new_tokens=256, return_full_text=False)

In [ ]:
outputs = base_model_pipe(create_instruction(riddle_dataset["test"][0], return_response=False), do_sample=True, max_new_tokens=256, temperature=0.1, top_k=50)

In [ ]:
outputs[0]["generated_text"]

'\n\nThe correct answer is B: candle.\n\nHere\'s how the description fits:\n\n* "My life can be measured in hours": A candle\'s life can be measured in hours, as it burns for a certain amount of time.\n* "I serve by being devoured": A candle serves by providing light, and it is "devoured" (burned) to do so.\n* "Thin, I am quick": A thin candle burns quickly.\n* "Fat, I am slow": A fat or large candle burns slowly.\n* "Wind is my foe": Wind can extinguish a candle\'s flame, making it a foe to the candle\'s life.\n\nWell done on creating a clever riddle!'

In [ ]:
riddle_dataset["test"][1]["question"]

"I am black when you buy me, red when you use me.  When I turn white, you know it's time to throw me away.  What am I?"

In [ ]:
for text, label in zip(riddle_dataset["test"][1]["choices"]["text"], riddle_dataset["test"][1]["choices"]["label"]):
  print(f"{label} : {text}")

A : charcoal
B : rose flower
C : ink
D : fruit
E : shoe


In [ ]:
outputs = base_model_pipe(create_instruction(riddle_dataset["test"][1], return_response=False), do_sample=True, max_new_tokens=256, temperature=0.1, top_k=50)

In [ ]:
outputs[0]["generated_text"]

'\n\nThe correct answer is A: charcoal.\n\nHere\'s how the description fits:\n\n* "I am black when you buy me": Charcoal is typically black when you purchase it.\n* "red when you use me": When you use charcoal, it turns red-hot as it burns.\n* "When I turn white, you know it\'s time to throw me away": As charcoal burns, it eventually turns white and becomes ash, indicating that it has been fully consumed and is no longer usable.'

Now we can set-up our LoRA configuration file - which will let the TRL library know how to create our LoRA adapters!

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=32,
    lora_alpha=64,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM",
)

### Fine-tuning!

Now onto the star of today's show: Fine-tuning!

We're going to use the `SFTTrainer` or "Supervised Fine-tuning Trainer" from the [TRL](https://github.com/huggingface/trl/tree/main) library today.

In essence, this is a trainer that will handle most of the fine-tuning process for us - including but not limited to:

- Dataset packing
- LoRA initialization
- Tokenizing

Let's set up some training hyper-parameters through transformers `TrainingArguments` class to get started. Here's a breakdown of which parameters are doing what:

- `outpur_dir` - indicates where we store the results of training locally
- `num_train_epochs` - how many epochs we will train for (somewhere ~3-4 is a good place to start)
- `per_device_train_batch_size` - how many batches we want per device. In this case, we only have one device - but we set this to a low value to keep memory consumption below 16GB GPU RAM
- `gradient_accumulation_steps` - this hyper-parameter lets us indicate how many steps we wish to accumulate our gradients for, this is a way to "cheat out" a larger batch size without extra memory consumption
- `gradient_checkpointing` - this lets us [trade off memory consumption for increased training time](https://huggingface.co/docs/transformers/v4.18.0/en/performance#gradient-checkpointing)
- `optim` - our optimizer! In this case, we're using  a fused ADAMW optimiser. The fused method is a faster version of the ADAMW optimizer but is reliant on CUDA (GPU). More information can be read [here](https://pytorch.org/docs/stable/generated/torch.optim.AdamW.html)

The rest of the hyper-parameters are taken directly from the QLoRA [paper](https://arxiv.org/abs/2305.14314) and are discussed in more detail there!

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="riddle-bot-v1",
    num_train_epochs=4,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    gradient_checkpointing=True,
    optim="adamw_torch_fused",
    save_strategy="epoch",
    learning_rate=2e-4,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="constant",
    push_to_hub=True,
)

Because we're going to automatically push our model to the hub, thanks to `push_to_hub=True`, we'll want to provide a Hugging Face Write token.

> NOTE: You can skip this step by commenting out `push_to_hub=True`

Now, finally, we can set-up our `SFTTrainer` which is going to help us fine-tune this model on our dataset we create at the beginning of the notebook!

We'll discuss a few parameters to clarify what they're doing:

- `formatting_func` - since we created a helper function to convert our dataset row into a Mistral-style Instruction prompt, we need to let TRL know to use it to create our prompts!
- `peft_config` - TRL will automatically load our model in LoRA format with this config.
- `packing` - this will let our model "pack" the context window to ensure we're not wasting precious memory on padding tokens where posssible
- `dataset_kwargs` - because we already added the special tokens to our prompts, we want to ensure we don't "re-add" them!

With those parameters set - we're clear for training!

In [ ]:
from trl import SFTTrainer

max_seq_length=2048

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=riddle_dataset["train"],
    formatting_func=create_instruction,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    dataset_kwargs={
        "add_special_tokens" : False,
        "append_concat_token" : False,
    }
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:440: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictionary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

All that's left to do is fine-tune our model!

In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.

TrainOutput(global_step=148, training_loss=1.1005052102578652, metrics={'train_runtime': 618.2408, 'train_samples_per_second': 0.964, 'train_steps_per_second': 0.239, 'total_flos': 5.492501954061926e+16, 'train_loss': 1.1005052102578652, 'epoch': 3.9466666666666668})

Now we can save it.

In [ ]:
trainer.save_model()

Let's clear up memory so we can do inference while staying under our memory budget.

In [ ]:
del model
del trainer
torch.cuda.empty_cache()

We'll need to load our mode back as a PEFT model, due to the use of LoRA, and then merge the LoRA layers back into the original model for use in Hugging Face pipelines.

In [ ]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    args.output_dir,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto"
)

merged_model = model.merge_and_unload()

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
merged_model.push_to_hub("ai-maker-space/riddle-bot-v1")

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/ai-maker-space/riddle-bot-v1/commit/26f17a8b53e17f52f0f5193228060acd24945380', commit_message='Upload LlamaForCausalLM', commit_description='', oid='26f17a8b53e17f52f0f5193228060acd24945380', pr_url=None, pr_revision=None, pr_num=None)

Now we can load our pipeline for `text-generation`.

In [ ]:
from transformers import pipeline

riddle_pipe = pipeline("text-generation", merged_model, tokenizer=tokenizer, max_new_tokens=256, return_full_text=False)

## Testing Fine-tuned Model

Now that we've fine-tuned, lets see how we did!

In [ ]:
riddle_dataset["test"][0]["question"]

'My life can be measured in hours. I serve by being devoured. Thin, I am quick. Fat, I am slow. Wind is my foe. What am I?'

In [ ]:
outputs = riddle_pipe(create_instruction(riddle_dataset["test"][0], return_response=False), do_sample=True, max_new_tokens=256, temperature=0.1, top_k=50)

In [ ]:
for text, label in zip(riddle_dataset["test"][0]["choices"]["text"], riddle_dataset["test"][0]["choices"]["label"]):
  print(f"{label} : {text}")

A : paper
B : candle
C : lamp
D : clock
E : worm


In [ ]:
outputs[0]["generated_text"]

'B'

Another example!

In [ ]:
riddle_dataset["test"][1]["question"]

"I am black when you buy me, red when you use me.  When I turn white, you know it's time to throw me away.  What am I?"

In [ ]:
outputs = riddle_pipe(create_instruction(riddle_dataset["test"][1], return_response=False), do_sample=True, max_new_tokens=256, temperature=0.1, top_k=50)

In [ ]:
for text, label in zip(riddle_dataset["test"][1]["choices"]["text"], riddle_dataset["test"][1]["choices"]["label"]):
  print(f"{label} : {text}")

A : charcoal
B : rose flower
C : ink
D : fruit
E : shoe


In [ ]:
outputs[0]["generated_text"]

'A'

Overall, our fine-tuning did a great job and allowed our model to generate our desired output - all with <16GB GPU memory, and 4 epochs of fine-tuning!